<a href="https://colab.research.google.com/github/Kweku21/waste_product_classification/blob/master/waste_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow-gpu


     |████████████████████████████████| 421.8MB 19kB/s 
     |████████████████████████████████| 3.8MB 34.6MB/s 
     |████████████████████████████████| 450kB 46.9MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [ ]:
pip install Keras==2.3.1

     |████████████████████████████████| 378kB 7.2MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle

data_dir = '/content/drive/My Drive/Dataset/Waste/Data'

gen = {
    "train": ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1. / 255,
        validation_split=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=30,
    ).flow_from_directory(
        directory=data_dir,
        target_size=(300, 300),
        subset='training',
    ),

    "valid": ImageDataGenerator(
        rescale=1 / 255,
        validation_split=0.1,
    ).flow_from_directory(
        directory=data_dir,
        target_size=(300, 300),
        subset='validation',
    ),
}





Using TensorFlow backend.


Found 2271 images belonging to 6 classes.
Found 250 images belonging to 6 classes.


In [ ]:
base_model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3))
base_model.load_weights('/content/drive/My Drive/Dataset/keras_models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')
])

opt = optimizers.nadam(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

batch_size = 200
epochs = 3000
train_generator = gen["train"]
valid_generator = gen["valid"]

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

filepath = "/content/drive/My Drive/Dataset/Waste/Model/model_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('/content/drive/My Drive/Dataset/Waste/Model/trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/3000
11/11 [==============================] - 254s 23s/step - loss: 1.6638 - accuracy: 0.2955 - val_loss: 1.5100 - val_accuracy: 0.3750

Epoch 00001: val_accuracy improved from -inf to 0.37500, saving model to /content/drive/My Drive/Dataset/Waste/Model/model_01-0.38.h5
Epoch 2/3000
11/11 [==============================] - 239s 22s/step - loss: 1.4546 - accuracy: 0.5000 - val_loss: 1.6166 - val_accuracy: 0.2812

Epoch 00002: val_accuracy did not improve from 0.37500
Epoch 3/3000
11/11 [==============================] - 231s 21s/step - loss: 1.2763 - accuracy: 0.5824 - val_loss: 1.1187 - val_accuracy: 0.6250

Epoch 00003: val_accuracy improved from 0.37500 to 0.62500, saving model to /content/drive/My Drive/Dataset/Waste/Model/model_03-0.62.h5
Epoch 4/3000


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 6 could not be retrieved. It could be because a worker has died.
  UserWarning)


11/11 [==============================] - 250s 23s/step - loss: 1.1760 - accuracy: 0.6023 - val_loss: 0.8919 - val_accuracy: 0.6875

Epoch 00004: val_accuracy improved from 0.62500 to 0.68750, saving model to /content/drive/My Drive/Dataset/Waste/Model/model_04-0.69.h5
Epoch 5/3000
11/11 [==============================] - 223s 20s/step - loss: 1.0894 - accuracy: 0.6353 - val_loss: 1.1488 - val_accuracy: 0.6250

Epoch 00005: val_accuracy did not improve from 0.68750
Epoch 6/3000
11/11 [==============================] - 236s 21s/step - loss: 1.0651 - accuracy: 0.6335 - val_loss: 0.9201 - val_accuracy: 0.6875

Epoch 00006: val_accuracy did not improve from 0.68750
Epoch 7/3000
11/11 [==============================] - 106s 10s/step - loss: 1.0314 - accuracy: 0.6193 - val_loss: 0.7420 - val_accuracy: 0.8125

Epoch 00007: val_accuracy improved from 0.68750 to 0.81250, saving model to /content/drive/My Drive/Dataset/Waste/Model/model_07-0.81.h5
Epoch 8/3000
11/11 [=============================